### Import and Setup

In [18]:
### Import libraries ###
import os
import tempfile
import PIL
from PIL import Image
import numpy as np
import streamlit as st
import streamlit.components.v1 as components
import utils
from streamlit_tags import st_tags, st_tags_sidebar

from llama_index.legacy.prompts import PromptTemplate
from llama_index.legacy.multi_modal_llms.gemini import GeminiMultiModal
from llama_index.legacy.schema import ImageDocument
from llama_index.legacy.multi_modal_llms.generic_utils import (
    load_image_urls,
)


In [17]:
#from utils.all_texts import * 

In [19]:
import sys
import importlib.util

file_path = '/Users/oksanapliushko/Desktop/PLI18542171_Thesis/ImaGenStories-main/utils/all_texts.py'

try:
    spec = importlib.util.spec_from_file_location("all_texts", file_path)
    all_texts = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(all_texts)
    
    print("Successfully imported all_texts")
    print("Contents of texts:", all_texts.texts.keys())
except Exception as e:
    print(f"Error importing all_texts: {e}")

Successfully imported all_texts
Contents of texts: dict_keys(['about', 'prompt_story'])


### Streamlit Configuration 

In [20]:
### Set config ###
st.set_page_config(
    page_title="ImaGenStories",
    page_icon="📝",
    layout="wide",
    initial_sidebar_state="auto",
    menu_items={
        'About': all_texts.texts["about"]
    }    
)

st.markdown(
    """
    <style>
    [aria-label="dialog"]{
        width: 90vw;
    }
    </style>
    """, unsafe_allow_html=True
)
st.title("🎨📝 ImaGenStories")

2024-09-17 12:54:44.864 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:54:44.865 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:54:44.866 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:54:44.867 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:54:44.867 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

### API Key Setup

In [21]:
### Keys ###
try:
    os.environ["GOOGLE_API_KEY"] = st.secrets["GOOGLE_API_KEY"]
except:
    st.error("Error: Secrets")

In [22]:
import streamlit as st
import os

# Access your actual secrets
st.write("GOOGLE_API_KEY is set:", "GOOGLE_API_KEY" in st.secrets)

# If you need to access it as an environment variable:
st.write(
    "GOOGLE_API_KEY environment variable is set:",
    "GOOGLE_API_KEY" in os.environ
)

2024-09-17 12:54:48.025 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:54:48.026 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:54:48.027 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:54:48.027 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:54:48.028 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:54:48.028 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:54:48.028 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:54:48.029 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

### Model and Global Variable Setup

In [25]:
### Models ###
gemini_vision = GeminiMultiModal(model_name="models/gemini-pro-vision")

### Global Variables ###
image = None
path = None
full_response = ""
message_placeholder = st.empty()
temp_dir = tempfile.mkdtemp()

# Access the prompt_story from the all_texts module
prompt_story = all_texts.texts["prompt_story"]
story_template = PromptTemplate(prompt_story)

2024-09-17 12:55:23.447 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:55:23.448 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


### Sidebar Configuration


In [26]:
st.sidebar.title("Parameters")
with st.sidebar:
    genre_select = st.selectbox(
        "Choose a genre",
        ("Adventure", "Mystery", "Police", "Romantic", "Science fiction", "Terror")
    )
    narrative_select = st.selectbox(
        "Choose a narrative type",
        ("Descriptive", "Linear", "Nonlinear", "Viewpoint")
    )
    language_select = st.selectbox(
        "Choose a language",
        ("English","Spanish")
    )
    size_select = st.selectbox(
        "Size of story",
        ("Short","Medium","Large")
    )

    keywords = st_tags_sidebar(
        label='Enter Keywords:',
        text='Press enter to add more',
        value=[],
        suggestions=['Other'],
        maxtags = 5,
        key='1')

2024-09-17 12:55:25.144 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:55:25.145 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:55:25.146 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:55:25.146 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:55:25.147 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:55:25.147 Session state does not function when running a script without `streamlit run`
2024-09-17 12:55:25.148 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:55:25.148 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:55

### Main Application Layout

In [29]:
tab_main, tab_info = st.tabs(["Main", "Info"])

with tab_main:
    left_column, right_column = st.columns(2)
    with left_column:
        uploaded_files = st.file_uploader("Choose a image file", type = (["jpg", "jpeg", "png"]),accept_multiple_files=True)
        for uploaded_file in uploaded_files:
            if uploaded_file is not None:
                image = Image.open(uploaded_file)
                with st.spinner('Loading...'):
                    st.image(image)
                path = os.path.join(temp_dir, uploaded_file.name)
                with open(path, "wb") as f:
                    f.write(uploaded_file.getvalue())

    with right_column:
        st.write("Upload some image, and press the button")
        if st.button('Generate a story!'):
            if path is None:
                st.warning("Error: Path not found")
            else:
                image_documents = [ImageDocument(image_path=os.path.join(temp_dir, name)) for name in os.listdir(temp_dir) if os.path.isfile(os.path.join(temp_dir, name))]
                message_placeholder = st.empty()
                full_response = ""
                prompt_final = story_template.format(genre=genre_select,narrative=narrative_select,language=language_select,size=size_select,tags=keywords)
                try:
                    with st.spinner('Loading...'):
                        for response in gemini_vision.stream_complete(prompt = prompt_final, image_documents=image_documents):
                            full_response += (response.text or "")
                            message_placeholder.markdown(full_response + "▌")
                        message_placeholder.markdown(full_response)
                        st.session_state.full_response = full_response
                except Exception as e:
                    st.error(e)
                
                # HTML component for copy button
                components.html('''
    <button class="button-13" onclick="copyContent()">📝 Copy</button>
    <script>
    function copyContent() {
        const content = document.querySelector('.stMarkdown').innerText;
        navigator.clipboard.writeText(content)
            .then(() => {
                console.log('Content copied to clipboard');
                // Optionally, you can show a success message to the user
            })
            .catch(err => {
                console.error('Failed to copy: ', err);
                // Optionally, you can show an error message to the user
            });
    }
    </script>
    <style>
    .button-13 {
        background-color: #fff;
        border: 1px solid #d5d9d9;
        border-radius: 8px;
        box-shadow: rgba(213, 217, 217, .5) 0 2px 5px 0;
        box-sizing: border-box;
        color: #0f1111;
        cursor: pointer;
        display: inline-block;
        font-family: "Amazon Ember",sans-serif;
        font-size: 13px;
        line-height: 29px;
        padding: 0 10px 0 11px;
        position: relative;
        text-align: center;
        text-decoration: none;
        user-select: none;
        -webkit-user-select: none;
        touch-action: manipulation;
        vertical-align: middle;
        width: 100px;
    }
    
    .button-13:hover {
        background-color: #f7fafa;
    }
    
    .button-13:focus {
        border-color: #008296;
        box-shadow: rgba(213, 217, 217, .5) 0 2px 5px 0;
        outline: 0;
    }
    </style>
    ''')

with tab_info:
    st.markdown(all_texts.texts["about"])

2024-09-17 12:56:54.280 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:56:54.282 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:56:54.283 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:56:54.285 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:56:54.286 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:56:54.286 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:56:54.287 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-17 12:56:54.287 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar